# Importing and Cleaning
### Optional: Google Colab

In [174]:
# if you're running in google colab, do this first
# from google.colab import drive 
# drive.mount('/content/drive/') # you will need to authenticate
# cd My\ Drive/ # cd into the directory where your data is stored, for me it's the My Drive folder

## Importing and Cleaning

In [175]:
import pandas as pd #libraries imported as we go for enhanced readability

In [176]:
# loading in our two datasets..we'll combine them later on
# if you load in the data set as I do, they'll need to be placed in the same directory
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [177]:
true.head() # notice reuters is in all the real articles, will need to extract it

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [178]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [179]:
# setting entire column to 0 so when we concat the dataframes we know which were real and which were fake
# In this case, the real articles are equal to 0
true["fake_news"] = 0 
fake["fake_news"] = 1

In [180]:
# examine the unique article subjects
fake["subject"].unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [181]:
true["subject"].unique() # dont care about these, probably will just use text and fake news score in classifier

array(['politicsNews', 'worldnews'], dtype=object)

## Extracting Text in Pandas

In [182]:
# we will have to standardize this column to match the other one so that our classifier is actually effective
just_text = true["text"]
just_text.head()

0    WASHINGTON (Reuters) - The head of a conservat...
1    WASHINGTON (Reuters) - Transgender people will...
2    WASHINGTON (Reuters) - The special counsel inv...
3    WASHINGTON (Reuters) - Trump campaign adviser ...
4    SEATTLE/WASHINGTON (Reuters) - President Donal...
Name: text, dtype: object

In [183]:
# do this with regular expressions. This will extract the text that comes after the hyphen that follows Reuters
just_text = just_text.str.extractall(r"^.* - (?P<text>.*)")

In [167]:
# notice multiIndex does not work with old data frame
# just_text.index

In [184]:
just_text = just_text.droplevel(1)
just_text.head() # works!

,text
0,The head of a conservative Republican faction ...
1,have issued rulings blocking Trump’s ban while...
2,The special counsel investigation of links bet...
3,Trump campaign adviser George Papadopoulos tol...
4,President Donald Trump called on the U.S. Post...


In [185]:
true = true.assign(text=just_text["text"]) # switcharoo on the columns
true.head()

,title,text,subject,date,fake_news
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,have issued rulings blocking Trump’s ban while...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...,politicsNews,"December 29, 2017",0


## Combining and Saving The Data

In [194]:
df = pd.concat([fake, true], axis = 0) # good to go for combine!

In [187]:
df = df.drop(["subject", "date", "title"], axis = 1) # we won't use these columns in our model

In [188]:
df.info() # odd, we have some empty text entries. We must remove the null records

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 21416
Data columns (total 2 columns):
text         44869 non-null object
fake_news    44898 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [189]:
df = df.dropna() # super simple in pandas!

In [190]:
df.info() # now we have a clean data frame!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44869 entries, 0 to 21416
Data columns (total 2 columns):
text         44869 non-null object
fake_news    44869 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [191]:
# optional: save the cleaned csv for other modeling
clean_text = df.to_csv("cleaned_news.csv", index = False)# prevents having two indices

Next steps are modeling!